<a href="https://colab.research.google.com/github/sushil4625/StockMarketAnalysis/blob/main/KPI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



--2023-03-08 17:08:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  73.2MB/s    in 0.2s    

2023-03-08 17:08:52 (73.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.8/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data= spark.read.option("header","True").csv("/content/Merged_Data1.csv")
data.show()
data.createOrReplaceTempView("data")
spark.sql("select * from data where sector is not NULL" ).show()
clean_data= spark.sql("select * from data where sector is not null" )
clean_data.show(5)
clean_data.createOrReplaceTempView("clean_data")
spark.sql("SELECT sector,FIRST(Open) as open_day, LAST(Close) as close_day from clean_data group by sector ").show()
spark.sql("SELECT Company,FIRST(Open) as open_day, LAST(Close) as close_day from clean_data group by Company").show()
spark.sql("SELECT Company,sector,FIRST(Open) as open_day, LAST(Close) as close_day from clean_data group by Company,sector ").show()
final_data= spark.sql("SELECT Company,sector,FIRST(Open) as open_day, LAST(Close) as close_day from clean_data group by Company,sector ")
firstKPI=final_data.withColumn("Percentage_Change",((final_data["close_day"]-final_data["open_day"])/final_data["open_day"])*100)
firstKPI.show(5)
firstKPI.createOrReplaceTempView("firstKPI")

+---+--------------------+--------------------+-------------+----------+-----+-----+-----+-----+---------+---------+------+-------+--------+---------------+---------------+---------------+--------------+
|_c0|              Sector|             Company|          BSE|      Date| Open| High|  Low|Close|Adj Close|   Volume|Low_52|High_52|PE_Ratio|  Total_Revenue|   Gross_Profit|     Market_Cap|   Net_Company|
+---+--------------------+--------------------+-------------+----------+-----+-----+-----+-----+---------+---------+------+-------+--------+---------------+---------------+---------------+--------------+
|  0|Pharmaceuticals &...|Aurobindo Pharma ...|AUROPHARMA.BO|02-01-2012|43.00|43.45|42.53|43.08|    39.92|113202.00|281.15|1023.10|   16.81|192259300000.00|105132900000.00|273374609408.00|23647300000.00|
|  1|Pharmaceuticals &...|Aurobindo Pharma ...|AUROPHARMA.BO|03-01-2012|43.53|45.88|43.45|45.50|    42.16|277525.00|281.15|1023.10|   16.81|192259300000.00|105132900000.00|273374609408

In [ ]:
## 1st KPI Above 5% growth company
spark.sql("select * from firstKPI where Percentage_Change>5").show()
KPI1=spark.sql("select * from firstKPI where Percentage_Change>5")
KPI1.coalesce(1).write.mode('overwrite').format("csv").save("KPI1.csv")

+--------------------+--------------------+--------+---------+------------------+
|             Company|              sector|open_day|close_day| Percentage_Change|
+--------------------+--------------------+--------+---------+------------------+
|    ABBOTT INDIA LTD|Pharmaceuticals &...| 1469.95|  1761.21|19.814279397258407|
|   ASHOK LEYLAND LTD|          Automobile|   23.00|   145.00| 530.4347826086956|
|Asian Paints Limited|Fast Moving Consu...|  258.00|  2828.20| 996.2015503875967|
|Aurobindo Pharma ...|Pharmaceuticals &...|   43.00|   466.50| 984.8837209302326|
|      BAJAJ AUTO LTD|          Automobile| 1595.00|  3733.85|134.09717868338558|
|BHARAT IMMUNOLOGI...|Pharmaceuticals &...|    7.98|    24.04|201.25313283208018|
|          BIOCON LTD|Pharmaceuticals &...|   45.90|   226.45| 393.3551198257081|
|BIOFIL CHEMICALS ...|Pharmaceuticals &...|    5.50|    44.68| 712.3636363636364|
|BRITANNIA INDUSTR...|Fast Moving Consu...|  222.50|  4414.35|1883.9775280898878|
|          CIPLA

In [ ]:
#2nd KPI Below 5% down company
spark.sql("select * from firstKPI where Percentage_Change<5").show()
KPI2=spark.sql("select * from firstKPI where Percentage_Change<5")
KPI2.coalesce(1).write.mode('overwrite').format("csv").save("KPI2.csv")

+------------------+----------+--------+---------+-------------------+
|           Company|    sector|open_day|close_day|  Percentage_Change|
+------------------+----------+--------+---------+-------------------+
|  Mindtree Limited|        IT|  957.50|   956.12|-0.1441253263707567|
|SCOOTERS INDIA LTD|Automobile|   28.70|    27.67|-3.5888501742160197|
+------------------+----------+--------+---------+-------------------+



In [ ]:
# 3rd KPI 5% growth company 
firstKPI.createOrReplaceTempView("firstKPI")
spark.sql("select * from firstKPI where Percentage_Change>5 ").show()
KPI3=spark.sql("select * from firstKPI where Percentage_Change>5")
KPI3.coalesce(1).write.mode('overwrite').format("csv").save("KPI3.csv")

+--------------------+--------------------+--------+---------+------------------+
|             Company|              sector|open_day|close_day| Percentage_Change|
+--------------------+--------------------+--------+---------+------------------+
|    ABBOTT INDIA LTD|Pharmaceuticals &...| 1469.95|  1761.21|19.814279397258407|
|   ASHOK LEYLAND LTD|          Automobile|   23.00|   145.00| 530.4347826086956|
|Asian Paints Limited|Fast Moving Consu...|  258.00|  2828.20| 996.2015503875967|
|Aurobindo Pharma ...|Pharmaceuticals &...|   43.00|   466.50| 984.8837209302326|
|      BAJAJ AUTO LTD|          Automobile| 1595.00|  3733.85|134.09717868338558|
|BHARAT IMMUNOLOGI...|Pharmaceuticals &...|    7.98|    24.04|201.25313283208018|
|          BIOCON LTD|Pharmaceuticals &...|   45.90|   226.45| 393.3551198257081|
|BIOFIL CHEMICALS ...|Pharmaceuticals &...|    5.50|    44.68| 712.3636363636364|
|BRITANNIA INDUSTR...|Fast Moving Consu...|  222.50|  4414.35|1883.9775280898878|
|          CIPLA

In [ ]:
# 4th KPI 5% down company
firstKPI.createOrReplaceTempView("firstKPI")
spark.sql("select * from firstKPI where Percentage_Change<5").show()
KPI4=spark.sql("select * from firstKPI where Percentage_Change<5")
KPI4.coalesce(1).write.mode('overwrite').format("csv").save("KPI4.csv")

+------------------+----------+--------+---------+-------------------+
|           Company|    sector|open_day|close_day|  Percentage_Change|
+------------------+----------+--------+---------+-------------------+
|  Mindtree Limited|        IT|  957.50|   956.12|-0.1441253263707567|
|SCOOTERS INDIA LTD|Automobile|   28.70|    27.67|-3.5888501742160197|
+------------------+----------+--------+---------+-------------------+



In [ ]:
# 5th KPI Sector-wise highest growth companies.
firstKPI.createOrReplaceTempView("firstKPI")
spark.sql("select * from (select *,row_number() over (partition by Sector order by Percentage_Change desc) as rn  from (select sector,company,round(Percentage_Change,2) as Percentage_Change from firstKPI where sector is not null)) where rn=1").show()
KPI5=spark.sql("select * from (select *,row_number() over (partition by Sector order by Percentage_Change desc) as rn  from (select sector,company,round(Percentage_Change,2) as Percentage_Change from firstKPI where sector is not null)) where rn=1 ")
KPI5.coalesce(1).write.mode('overwrite').format("csv").save("KPI5.csv")

+--------------------+--------------------+-----------------+---+
|              sector|             company|Percentage_Change| rn|
+--------------------+--------------------+-----------------+---+
|          Automobile|   EICHER MOTORS LTD|          2011.57|  1|
|Fast Moving Consu...|BRITANNIA INDUSTR...|          1883.98|  1|
|                  IT|      TATA ELXSI LTD|          2428.32|  1|
|Pharmaceuticals &...|NATURAL CAPSULES ...|           1047.8|  1|
+--------------------+--------------------+-----------------+---+



In [ ]:
# 6th KPI Sector-wise lowest growth companies.
firstKPI.createOrReplaceTempView("firstKPI")
spark.sql("select * from (select *,row_number() over (partition by Sector order by Percentage_Change ) as rn from (select sector,company,round(Percentage_Change,2)as Percentage_Change from firstKPI where sector is not null and Percentage_Change is not null )) where rn=1").show()
KPI6=spark.sql("select * from (select *,row_number() over (partition by Sector order by Percentage_Change ) as rn  from (select sector,company,round(Percentage_Change,2)as Percentage_Change from firstKPI where sector is not null and Percentage_Change is not null )) where rn=1")
KPI6.coalesce(1).write.mode('overwrite').format("csv").save("KPI6.csv")

+--------------------+--------------------+-----------------+---+
|              sector|             company|Percentage_Change| rn|
+--------------------+--------------------+-----------------+---+
|          Automobile|  SCOOTERS INDIA LTD|            -3.59|  1|
|Fast Moving Consu...|             ITC LTD|           184.34|  1|
|                  IT|    Mindtree Limited|            -0.14|  1|
|Pharmaceuticals &...|VISTA PHARMACEUTI...|             7.47|  1|
+--------------------+--------------------+-----------------+---+

